In [ ]:
import os
import cv2
import mediapipe as mp
import pandas as pd
from tqdm import tqdm

# Rutas
INPUT_DIR = '../../data/dataset_raw'  # Carpeta con videos
OUTPUT_DIR = '../dataset_landmarks'  # Carpeta donde se guardarán los CSV

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Inicializar MediaPipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)
mp_drawing = mp.solutions.drawing_utils

# Función para extraer landmarks de un solo frame
def extract_landmarks(results):
    if not results.pose_landmarks:
        return None
    row = []
    for lm in results.pose_landmarks.landmark:
        row.extend([lm.x, lm.y, lm.z, lm.visibility])
    return row

# Procesar todos los videos
for file in tqdm(os.listdir(INPUT_DIR)):
    if not file.endswith(".mp4"):
        continue

    video_path = os.path.join(INPUT_DIR, file)
    cap = cv2.VideoCapture(video_path)

    filename = os.path.splitext(file)[0]
    label = "_".join(filename.split("_")[2:4])  # e.g., "caminar_hacia"

    landmarks_list = []
    frame_id = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        landmarks = extract_landmarks(results)
        if landmarks:
            landmarks_list.append([frame_id, label] + landmarks)

        frame_id += 1

    cap.release()

    # Guardar en CSV
    if landmarks_list:
        df = pd.DataFrame(landmarks_list)
        num_landmarks = int((len(df.columns) - 2) / 4)
        col_names = ["frame", "label"] + [
            f"{joint}_{coord}"
            for joint in range(num_landmarks)
            for coord in ['x', 'y', 'z', 'vis']
        ]
        df.columns = col_names
        df.to_csv(os.path.join(OUTPUT_DIR, filename + ".csv"), index=False)


Procesando videos y extrayendo landmarks...


FileNotFoundError: [WinError 3] The system cannot find the path specified: '../../data/dataset_raw'

: 